In [1]:
import pandas as pd
from sklearn.linear_model import Ridge, Lasso
import numpy as np
from tqdm.notebook import tqdm
import sys
import os
import re
import win32com.client
import os

In [2]:
# python : 3.7.3
# pandas : 0.25.3
# numpy : 1.16.2
# sklearn : 0.21.2
# tqdm : 4.38.0

### Read csv and search target columns

In [3]:
data = pd.read_excel('Privatized Data for Evaluation.xlsx').T

In [4]:
tmp = data.loc[['Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7'], :].T.values
target_cols = []
name_cols = []
for i,(pr, var, name) in enumerate(tmp):
        if var in ['Average revenue per new customer - Falcon', 'Average revenue per existing customer - Falcon',
                  'Gross Adds - Falcon(Norm)', 'Net Migrations - Falcon(Norm)']:
            target_cols += [i]
            name_cols += [name]

In [5]:
# I AM ADD 12 ROW IN END BECAUSE IN DATASET WE NEED PREDICTION ON THIS TIME
def change_year(x):
    new_x = x.split('.')[0]
    x_1 = int(new_x.split('/')[0]) + 1
    x_2 = int(new_x.split('/')[1]) + 1
    return str(x_1) + '/' + str(x_2)
    

add_pred = data.iloc[-12:, :].copy()
add_pred.index = [change_year(x) + x[5:] for x in add_pred.index]
add_pred.iloc[: ,2:] = 'x'
add_pred = add_pred.iloc[:6, :]
data = pd.concat([data, add_pred], axis = 0)

### Split index and train (past values where no target data) and test (values that we need predict)

In [6]:
dict_train_index = {}
for target in target_cols:
    train_data_index = [i for i, (ind, val) in enumerate(zip(data.index, data[target])) if (re.match('\d+', ind) ) and val == val]
    test_data_index = train_data_index[-12:]
    train_data_index = train_data_index[:-12]
    #     test_data_index = [i for i, (ind, val) in enumerate(zip(data.index, data[target])) if re.match('\d+', ind) and val == 'x']
    dict_train_index[target] = (train_data_index, test_data_index)

### Check that split is correct

In [7]:
data.head(10)

0                         1  \
Anonymised Variables         10             Generic Group   
Unnamed: 1                   11             Generic Brand   
Unnamed: 2                   12  Generic Product Category   
Unnamed: 3                   13          Generic Product    
Unnamed: 4                  NaN    Generic Group variable   
Unnamed: 5                   14          Generic Variable   
Unnamed: 6                   15      Generic Sub-Variable   
Unnamed: 7                   16         Generic LookupKey   
Yearly                Quarterly                     Units   
16/17                        Q1       2016-04-01 00:00:00   

                                                                      2  \
Anonymised Variables                                          Segment 2   
Unnamed: 1                Sandesh Brand 1 / Sandesh Brand 2 - Segment 2   
Unnamed: 2                                                    Broadband   
Unnamed: 3                                                     Tortoise   
Unnamed: 4                                                         ARPU   
Unnamed: 5             Average revenue per existing customer - Tortoise   
Unnamed: 6                                          Intentionally Blank   
Unnamed: 7            Segment 2Sandesh Brand 1 / Sandesh Brand 2 - S...   
Yearly                                                           Pounds   
16/17                                                          -1.65976   

                                                                      3  \
Anonymised Variables                                          Segment 2   
Unnamed: 1                Sandesh Brand 1 / Sandesh Brand 2 - Segment 2   
Unnamed: 2                                                    Broadband   
Unnamed: 3                                                     Tortoise   
Unnamed: 4                                                         ARPU   
Unnamed: 5                  Average revenue per new customer - Tortoise   
Unnamed: 6                                          Intentionally Blank   
Unnamed: 7            Segment 2Sandesh Brand 1 / Sandesh Brand 2 - S...   
Yearly                                                           Pounds   
16/17                                                         -0.501863   

                                                                      4  \
Anonymised Variables                                          Segment 2   
Unnamed: 1                Sandesh Brand 1 / Sandesh Brand 2 - Segment 2   
Unnamed: 2                                                    Broadband   
Unnamed: 3                                                     Tortoise   
Unnamed: 4                                                      Revenue   
Unnamed: 5                                           Revenue - Tortoise   
Unnamed: 6                                          Intentionally Blank   
Unnamed: 7            Segment 2Sandesh Brand 1 / Sandesh Brand 2 - S...   
Yearly                                                               £m   
16/17                                                           1.27157   

                                                                      5  \
Anonymised Variables                                          Segment 2   
Unnamed: 1                Sandesh Brand 1 / Sandesh Brand 2 - Segment 2   
Unnamed: 2                                                    Broadband   
Unnamed: 3                                                     Tortoise   
Unnamed: 4                                                       Volume   
Unnamed: 5                                          Net Adds - Tortoise   
Unnamed: 6                                          Intentionally Blank   
Unnamed: 7            Segment 2Sandesh Brand 1 / Sandesh Brand 2 - S...   
Yearly                                                           (000)s   
16/17                                                         -0.490581   

                                                      

In [8]:
data.iloc[dict_train_index[8][1], :].head(5)

0                    1          2         3        4         5  \
19/20.6   Q3  2019-10-01 00:00:00   0.162659 -0.852943 -1.23583 -0.157975   
19/20.7   Q3  2019-11-01 00:00:00   0.329939 -0.932966 -1.31214 -0.943668   
19/20.8   Q3  2019-12-01 00:00:00  0.0865817  -1.06312 -1.39208   1.02764   
19/20.9   Q4  2020-01-01 00:00:00  -0.093443 -0.708625 -1.45966   1.14815   
19/20.10  Q4  2020-02-01 00:00:00  0.0331674 -0.252834 -1.53628 -0.746311   

                6        7        8          9       10        11        12  \
19/20.6  -1.23125 -1.23909 -1.65166  0.0091798  1.12336   1.04397  0.110644   
19/20.7  -1.30681 -1.33438 -1.44252  0.0338353  1.22499   1.14528  0.457847   
19/20.8   -1.4018 -1.38078 -1.75421   -0.22804  1.27101   1.12022 -0.702604   
19/20.9  -1.44805 -1.42419 -1.34523  -0.519837  1.29871   1.06973    -1.085   
19/20.10 -1.49132 -1.51459 -1.67176   -1.40362  1.41539  0.942558  0.772069   

               13        14         15         16       17       18        19  
19/20.6   1.09295 -0.854639    1.23378   0.394393  1.80658 -1.38034   1.09179  
19/20.7   1.17604  -0.76564    1.70277   0.536026  1.74941 -1.01014   1.67145  
19/20.8   1.27538  -1.12423  0.0878356  0.0641501  1.27133 -1.76499   1.01867  
19/20.9   1.32037 -0.877555    1.38314    1.36216  2.43961 -1.14266   2.03412  
19/20.10  1.34745  -1.34638   0.215232  -0.150596  1.65914 -2.12074  0.609747

### Choose train cols and translate the values into the float from string

In [9]:
train_cols = list(range(data.shape[1]))[:-1]
data[0] = [int(x[1]) if type(x) == str and len(x) == 2 and x[0] == 'Q' else x for x in data[0].tolist()]

In [10]:
def clear_val(val):
    if type(val) != str:
        return val
    elif ',' not in val:
        return float(val)
    else:
        return float(re.sub(',', '.', val))

In [11]:
for col in train_cols:
    data.iloc[train_data_index, col] = [clear_val(x) for x in data.iloc[train_data_index, col].values]

### Compute featrues for Linear Regression

In [12]:
train_data = pd.DataFrame()
quater_col = 0
for col in tqdm(target_cols):
    list_last = []
    list_mean = []
    list_max = []
    quater_list = data.iloc[train_data_index + test_data_index, quater_col].tolist()
#     for ever columns for evey point compute different values in that quarter in past years
    for i in range(len(train_data_index) - 28, len(train_data_index) + len(test_data_index) ):
        tmp_data = data.iloc[train_data_index[:i - 9], [quater_col , col]].copy().fillna(0)
        tmp_values = tmp_data[tmp_data[quater_col] == quater_list[i]][col].tolist()
        list_last += [tmp_values[-1]]
        list_mean += [np.nanmean(tmp_values)]
        list_max += [np.nanmax(tmp_values)]

    train_data['last_{}'.format(col)] = list_last
    train_data['mean_{}'.format(col)] = list_mean
    train_data['max_{}'.format(col)] = list_max
#  fill NA by last value
train_data['range'] = [i + 1 for i in range(train_data.shape[0])]
train_data = train_data.fillna(method = 'bfill').fillna(0)

### Function for different types of predictions

In [13]:
def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) 

In [104]:
len_train = len(train_data_index)
num_periods = len(test_data_index)

In [105]:
def predict_arpnew(X_train, y_train, X_test, alpha = 0.1, features = [12,  0,  5 , 4  ,6]):
    X_train = X_train[-8:, features]
    y_train = y_train[-8:]
    clf = Ridge(alpha, random_state = 228)
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test[:, features])
    return 0.7 * y_predict

In [106]:
def predict_arpex(X_train, y_train, X_test, alpha = 0.1):
    X_train = X_train[-22:]
    y_train = y_train[-22:]
    clf = Ridge(alpha, random_state = 228)
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    return y_predict + 1.15 * (y_train[-1] - y_predict[0])

In [107]:
def predict_grossadd(y_train, len_predict):
    y_train = y_train[-13:]
    return np.array(len_predict * [ np.median(y_train) ]) 

In [108]:
def predict_netmigr(y_train, len_predict):
    y_train = y_train[-5:]
    return np.array(len_predict * [ np.median(y_train) ]) * 1.3

### Train and predict targets

In [109]:
start_index = 15
X_train = train_data.values[-( len_train + num_periods - start_index):  -num_periods]
X_test = train_data.values[-num_periods:].copy()

target_col = target_cols[0]
y_train = data.iloc[train_data_index[start_index:], target_col]
y_predict_0 = predict_arpnew(X_train, y_train, X_test)

target_col = target_cols[1]
y_train = data.iloc[train_data_index[start_index:], target_col]
y_predict_1 = predict_arpex(X_train, y_train, X_test)

target_col = target_cols[2]
y_train = data.iloc[train_data_index[start_index:], target_col]
y_predict_2 = predict_grossadd(y_train, 12)

target_col = target_cols[3]
y_train = data.iloc[train_data_index[start_index:], target_col]
y_predict_3 = predict_netmigr(y_train, 12)

In [110]:
predict = pd.DataFrame()
predict['Time Period'] = [x[:7] for x in data[1].astype('str').tolist()[-12:]]
predict['Time Period'][6:] = ['2020' + x[4:] for x in predict['Time Period'][6:].values]

predict[name_cols[0]] = y_predict_0
predict[name_cols[1]] = y_predict_1
predict[name_cols[2]] = y_predict_2
predict[name_cols[3]] = y_predict_3
    
predict.to_csv('predict.csv', index = None)

## COMPUTE ROBOTNESS

In [112]:
times = pd.to_datetime(data[1].iloc[9:]).reset_index(drop = True)

In [114]:
num_iter = 12
num_periods = 12

start_date_for = 'April 2019'
ind_start = times.index[-1] - times[times == start_date_for].index[0]

end_date_for = 'March 2020'
ind_end = times.index[-1] - times[times == end_date_for].index[0]


all_predictions = {}
for shift_day in range(num_iter):
    print(shift_day)
    
    X_train = train_data.values[-( len_train + num_periods - start_index):  -num_periods - shift_day - ind_end].copy()
    X_test = train_data.values[-num_periods - shift_day - ind_end:-  shift_day - ind_end].copy()
    
    target_col = target_cols[0]
    y_train = data.iloc[train_data_index[start_index:-shift_day - ind_end], target_col]
    y_predict_0 = predict_arpnew(X_train, y_train, X_test)
    
    target_col = target_cols[1]
    y_train = data.iloc[train_data_index[start_index:-shift_day - ind_end], target_col]
    y_predict_1 = predict_arpex(X_train, y_train, X_test)

    target_col = target_cols[2]
    y_train = data.iloc[train_data_index[start_index:-shift_day - ind_end], target_col]
    y_predict_2 = predict_grossadd(y_train, 12)

    target_col = target_cols[3]
    y_train = data.iloc[train_data_index[start_index:-shift_day - ind_end], target_col]
    y_predict_3 = predict_netmigr(y_train, 12)
        
    predict = pd.DataFrame()
    predict['Time Period'] = [x[:7] for x in data[1].astype('str').tolist()[-12:]]
    predict['Time Period'][6:] = ['2020' + x[4:] for x in predict['Time Period'][6:].values]

    predict[name_cols[0]] = y_predict_0
    predict[name_cols[1]] = y_predict_1
    predict[name_cols[2]] = y_predict_2
    predict[name_cols[3]] = y_predict_3
    
    path = os.path.join('robust', 'submission{}.csv'.format(shift_day + 1))
    predict.to_csv(path, index = None)

0
1
2
3
4
5
6
7
8
9
10
11
